In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
try:
    from . import contants
except ImportError:
    import constants

import re
import logging
import shutil
import tempfile
import unicodedata
import datetime
import csv
import sys

from pathlib import Path

from humanfriendly import prompts, 
import textwrap




In [3]:
from library import GDrive
from library.GDrive import GDriveError

In [4]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [5]:
def do_exit(exit_level=0, message='', testing=False):

    if testing:
        print(f'TESTING--EXIT MESSAGE: ({exit_level}){message}')
    elif exit_level > 0:
        logger.warning(f'exiting before completion with exit code {exit_level}: {message}')
        print('exiting due to errors:')
        print(message)
        sys.exit(exit_level)
    else:
        print(message)
        sys.exit(0)
        

In [6]:
def slugify(value, allow_unicode=False, lower=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    if lower:
        value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '_', value).strip('-_')

In [7]:
def recurse_folders(drive, parents, 
                    fields=['parents', 'id', 'name', 'mimeType', 'owners', 'modifiedTime', 'webViewLink', 'parents'], 
                    file_list=[], skipped=[], depth=0):
    '''recursively find all files in a google drive folder'''
    if depth == 0:
        file_list = []
        skipped = []
    logger.info(f'recursion depth: {depth}')
    try:
        result = drive.search(parents=parents, fields=fields)
    except GDriveError as e:
        logger.error(f'error accessing google drive: {e}')
        skipped.append(parents)
        result = {}
        
    for f in result:
        if drive.MIMETYPES['folder'] == f.get('mimeType'):
            return_files, return_skipped = recurse_folders(drive=drive, parents=f['id'], 
                                         fields=fields, 
                                         file_list=file_list,
                                         skipped=skipped,
                                         depth=depth+1)
            file_list + return_files
            skipped + return_files
        else:
            file_list.append(f)
    return (file_list, skipped)
            

In [35]:
def get_folder_id():
    '''get folder id from URL provided by user
    
    Returns:
        str: unique google drive folder id'''
    valid_url = False
    retry = 4
    folderID = None
    print('\n')
    while not valid_url and retry > 0:
        retry -= 1
        url = prompts.prompt_for_input('Paste the URL of a Google Drive folder to audit:\n')
        
        match = re.match("https:\/\/drive\.google\.com(?:\/\w+)+\/(\w+)(?:\?\S+)?$", url)
        if not match:
            print('That does not appear to be a valid google drive folder URL')
            continue
        else:
            folderID = match[1]
            valid_url = True
    
    return folderID

In [37]:
def get_secrets():
    if not constants.SECRETS_PATHS['saved'].exists():
        constants.SECRETS_PATHS['saved'].parent.mkdir(parents=True, exist_ok=True)
        
    if not constants.SECRETS_PATHS['saved'].exists():
        try:
            logger.debug(f'trying to move secrets from {constants.SECRETS_PATHS["downloaded"]}')
            shutil.move(constants.SECRETS_PATHS['downloaded'], constants.SECRETS_PATHS['saved'])
        except FileNotFoundError as e:
            do_exit(1, "client_secrets.json file is not avialable. Download the secrets file (see documentation) and try again.")
        except OSError as e:
            do_exit(1, f"failed to install secrets file: {e}")
    else:
        logger.debug(f'using {constants.SECRETS_PATHS["saved"]}')
        
    return constants.SECRETS_PATHS['saved']

In [26]:
def output_csv(name, file_list, output=None):    
    if output:
        output = Path(output)/name
        if not output.parent.exists():
            output.parent.mkdir(exist_ok=True, parents=True)
    else:
        temp_dir = tempfile.mkdtemp()
        output = Path(temp_dir)/name
    logger.debug(f'using output file: {output}')
    
    fieldnames = ['name', 'emailAddress', 'displayName', 'webViewLink', 'modifiedTime', 'mimeType']
    pull = ['name', 'webViewLink', 'modifiedTime', 'mimeType']
    
    try:
        with open(output, 'w') as csvfile:            
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in file_list:
                line = {}
                for n in pull:
                    line[n] = row.get(n, '?error?')
                line['displayName'] = row.get('owners', [{}])[0].get('displayName', 'Unknown')
                line['emailAddress'] = row.get('owners', [{}])[0].get('emailAddress', 'Unknown')
                writer.writerow(line)
    except OSError as e:
        do_exit(1, f"error writing {output}: {e}")

    return output

In [41]:
def main():
    
    secrets = get_secrets()
    
    drive = GDrive(secrets=secrets, scopes=constants.SCOPES)
    
#     return drive
    
    wrapper = textwrap.TextWrapper(replace_whitespace=True, drop_whitespace=True, width=60)
    welcome = f'{constants.APP_NAME} provides a Google Sheet that shows all of the files and owners within a google drive folder.'
    print('\n'.join(wrapper.wrap(welcome)))
    
    
    while True:
        folder_id = get_folder_id()

        if not folder_id:
            do_exit(1, 'No valid folder URL was provided. Exiting')

        try:
            folder_props = drive.get_properties(fileId=folder_id, fields=['owners', 'name', 'webViewLink', 'kind', 'id', 'mimeType'])
        except GDriveError as e:
            msg = f'failed to get information on this folder: {e}'
            do_exit(1, msg)


        folder_name = folder_props.get('name', 'UNKNOWN FOLDER')
        time_string = datetime.date.strftime(datetime.datetime.now(), '%Y%m%d_%H%M%S')
        owners = folder_props.get('owners', [{}])

        if len(owners) > 0:
            owner_name = owners[0].get('displayName', 'UNKNOWN OWNER')
        else:
            owner_name = "UNKNOWN OWNER"                           

        google_filename = f'FOLDER:{folder_name} OWNER:{owner_name}-ownership audit [{time_string}]'
        local_filename = f'{time_string}.csv'

        files, skipped = recurse_folders(drive, parents=folder_id)

        local_file = output_csv(local_filename, files)
        try:
            logger.debug(f'sending csv to google sheets: {google_filename}')
            added_file = drive.add_file(local_file, name=google_filename, target_mimeType='sheets')
        except GDriveError as e:
            do_exit(1, f'failed to send audit sheet to google: {e}\nLocal copy available here: {local_file}')


        print(f'Audit completed and stored in the sheet below.')
        print(f'Audit of folder: {added_file.get("webViewLink", "ERROR")}\n\n')
        print(f'Use the sheet linked below to assist in viewing and filtering the audit:')
        print(constants.AUDIT_FILTER)
        
        cont = prompts.prompt_for_confirmation('Audit another Folder?', default=False)
        if not cont:
            break

    
    return added_file
   


In [43]:
if __name__ == "__main__":
    f = main()

DEBUG:__main__:using /Users/aciuffo/.cache/gdrive_audit/client_secrets.json
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


gdrive_audit provides a Google Sheet that shows all of the
files and owners within a google drive folder.



 Paste the URL of a Google Drive folder to audit:
 https://drive.google.com/drive/folders/0B5phaBV4N75ja3YzdUJUcTljTnc?resourcekey=0-mrSF7VJLpK31J7sQbdWWmw



DEBUG:library.GDrive:attempt: 0
DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v3/files/0B5phaBV4N75ja3YzdUJUcTljTnc?fields=kind%2Cname%2CmimeType%2Cid%2Cowners%2CwebViewLink&supportsAllDrives=true&alt=json
INFO:__main__:recursion depth: 0
DEBUG:library.GDrive:attempt: 0
DEBUG:library.GDrive:QUERY STRING: "0B5phaBV4N75ja3YzdUJUcTljTnc" in parents
DEBUG:library.GDrive:fettching page of 300 results with query "0B5phaBV4N75ja3YzdUJUcTljTnc" in parents
DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v3/files?q=%220B5phaBV4N75ja3YzdUJUcTljTnc%22+in+parents&corpora=user&includeItemsFromAllDrives=false&supportsAllDrives=false&fields=nextPageToken%2C+files%28id%2Cowners%2CmodifiedTime%2CwebViewLink%2Cparents%2Cname%2CmimeType%29&driveId=&pageSize=300&pageToken=&alt=json
DEBUG:library.GDrive:2 total matches returned
DEBUG:__main__:using output file: /var/folders/b0/67fplmbs6h38d0yjvvwkt898z4pk2_/T/tmp4tf4tq_l/

Audit completed and stored in the sheet below.
Audit of folder: https://docs.google.com/spreadsheets/d/1kmGwO1ysJ_0AGvjWyHcRTAlSW1AUne1qJxBRhHCabWw/edit?usp=drivesdk


Use the sheet linked below to assist in viewing and filtering the audit:
https://docs.google.com/spreadsheets/d/15U7uA7O9yAX61g_WPvdeGGuEninmJbnnyzA7wbHIaw0/copy

 Audit another Folder? y/N [y/N] y



DEBUG:humanfriendly.prompts:Confirmation granted by reply ('y').





 Paste the URL of a Google Drive folder to audit:
 https://drive.google.com/drive/folders/0B5phaBV4N75ja3YzdUJUcTljTnc?resourcekey=0-mrSF7VJLpK31J7sQbdWWmw



DEBUG:library.GDrive:attempt: 0
DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v3/files/0B5phaBV4N75ja3YzdUJUcTljTnc?fields=kind%2Cname%2CmimeType%2Cid%2Cowners%2CwebViewLink&supportsAllDrives=true&alt=json
INFO:__main__:recursion depth: 0
DEBUG:library.GDrive:attempt: 0
DEBUG:library.GDrive:QUERY STRING: "0B5phaBV4N75ja3YzdUJUcTljTnc" in parents
DEBUG:library.GDrive:fettching page of 300 results with query "0B5phaBV4N75ja3YzdUJUcTljTnc" in parents
DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v3/files?q=%220B5phaBV4N75ja3YzdUJUcTljTnc%22+in+parents&corpora=user&includeItemsFromAllDrives=false&supportsAllDrives=false&fields=nextPageToken%2C+files%28id%2Cowners%2CmodifiedTime%2CwebViewLink%2Cparents%2Cname%2CmimeType%29&driveId=&pageSize=300&pageToken=&alt=json
DEBUG:library.GDrive:2 total matches returned
DEBUG:__main__:using output file: /var/folders/b0/67fplmbs6h38d0yjvvwkt898z4pk2_/T/tmpawe9yc2f/

Audit completed and stored in the sheet below.
Audit of folder: https://docs.google.com/spreadsheets/d/1A8sAN0c_aRqRm2nb5wO6GJu7yJPxDuRbvcF7Wqva5tQ/edit?usp=drivesdk


Use the sheet linked below to assist in viewing and filtering the audit:
https://docs.google.com/spreadsheets/d/15U7uA7O9yAX61g_WPvdeGGuEninmJbnnyzA7wbHIaw0/copy

 Audit another Folder? y/N [y/N] n



DEBUG:humanfriendly.prompts:Confirmation denied by reply ('n').


In [44]:
!jupyter-nbconvert --to python --template python_clean gdrive_audit.ipynb

[NbConvertApp] Converting notebook gdrive_audit.ipynb to python
[NbConvertApp] Writing 7794 bytes to gdrive_audit.py
